# Install Package

In [41]:
! pip install langchain langchain_experimental langchain-openai langchain-qdrant pypdf sentence-transformers


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
from langchain_openai import OpenAIEmbeddings

embedding_models = OpenAIEmbeddings(
    openai_api_base = "http://localhost:1234/v1",
    api_key="type-anything-here",
    check_embedding_ctx_length=False
)

In [43]:
query = "This is my first embedding input"
vector = embedding_models.embed_query(query)
print(len(vector), vector)

768 [-0.027142411097884178, 0.06298010796308517, -0.13541874289512634, -0.07841941714286804, 0.009512169286608696, -0.026087282225489616, 0.09167490899562836, 0.006524229887872934, 0.003317721653729677, 0.007525214459747076, -0.038597945123910904, 0.013800815679132938, 0.0630827248096466, 0.07078425586223602, -0.03753889352083206, 0.037601538002491, 0.05013149604201317, -0.09621778130531311, -0.0027343661058694124, 0.029445834457874298, -0.03486834093928337, -0.03419571369886398, -0.03891743719577789, 0.00031672007753513753, 0.03477725759148598, 0.014772550202906132, 0.03315872699022293, -0.004717223346233368, -0.06687022745609283, 0.022569363936781883, 0.01789688505232334, 0.008520593866705894, 0.020647093653678894, -0.03244689851999283, -0.008830457925796509, 0.011679173447191715, 0.01600640080869198, 0.038365110754966736, 0.018955811858177185, -0.05251629650592804, -0.007539879996329546, -0.0074018859304487705, 0.02607267163693905, 0.00463831564411521, -0.03355007246136665, 0.014504

In [44]:
from pypdf import PdfReader
file_path = "./Sách Deep Learning cơ bản - v2.pdf"
reader = PdfReader(file_path)
reader.pages[30].extract_text()

'1.3 Hướng dẫn cài đặt anaconda 31\n1.3.4 Hướng dẫn sử dụng Jupyter notebook\nHình 1.9: Bên trái: Mở jupyter notebook trên win10, bên phải: Mở trên win7\nGiao diện mở ra, click vào new chọn Python3 để tạo 1 notebook mới\nSau đấy thì dùng giống như google colab, nhưng điểm khác biệt là jupyter notebook sẽ chạy trên\nmáy của bạn chứ không phải cloud như google colab.'

In [45]:
from langchain_experimental.text_splitter import SemanticChunker
text_spliter = SemanticChunker(embedding_models)

docs = text_spliter.create_documents([page.extract_text() for page in reader.pages])
# docs = text_spliter.create_documents()
print(len(docs))
print(docs[0]) 


877
page_content=''


In [46]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [47]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [48]:
client = QdrantClient(":memory:")

In [49]:
client.create_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [50]:
docs[0]

Document(metadata={}, page_content='')

In [51]:
client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc.page_content).tolist(), payload=doc.to_json()
        )
        for idx, doc in enumerate(docs)
    ],
)

In [54]:
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode("Python cơ bản").tolist(),
    limit=3,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'document', 'Document'], 'kwargs': {'page_content': '. 31\n2 Python cơ bản . .', 'type': 'Document'}} score: 0.8512605046234447
{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'document', 'Document'], 'kwargs': {'page_content': '38 Chương 2. Python cơ bản\nprint(c) # Prints "[[ 7.', 'type': 'Document'}} score: 0.6732336235684331
{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'document', 'Document'], 'kwargs': {'page_content': '. 40\n2.6 Matplotlib 40\nII Machine learning cơ bản\n3 Linear regression . .', 'type': 'Document'}} score: 0.45987344652287504
